In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import progressbar as progressbar

%load_ext autoreload
%autoreload 2

In [2]:
print(f'TF version: {tf.__version__}')
tf.config.list_physical_devices()

TF version: 2.2.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

# Read Data

In [4]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '../datasets/'

## Amazon Fashion

In [5]:
data_path = 'data/Amazon/'
# file_name = 'Amazon_full' # file_name = 'Amazon_05_users' 
# file_name = 'Amazon_01_users'
# file_name = 'am_80k_users'
file_name = 'am_40k_users'

## MovieLens

In [85]:
# data_path = 'data/ML/'
# file_name = 'ml_1m'
# file_name = 'ML_full' # file_name = 'ML_05_users'
# file_name = 'ML_01_users'

In [7]:
df = pd.read_pickle(path + data_path + file_name)
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes
df.head()

,user,item,datetime,rating,user_id,item_id
1845392,AGYA6NBNU1VUH,B0020B4EYU,2015-04-14,3.0,34371,11002
919114,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
5872323,AGYA6NBNU1VUH,B00KR6I6IK,2015-04-14,5.0,34371,75683
1237107,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
9971633,AGYA6NBNU1VUH,B008GXBRNM,2015-12-20,5.0,34371,35964


### Create ML split (paper)

In [7]:
test_set = pd.DataFrame()
train_set = pd.DataFrame()
pbar = progressbar.ProgressBar()
for u in pbar(df.user_id.unique()):
    last_item = df[df.user_id==u].iloc[-1:]
    test_set = test_set.append(last_item)
    remaining = df[df.user_id==u].iloc[:-1]
    train_set = pd.concat([train_set, remaining])

100% |########################################################################|


In [87]:
test_set = pd.read_pickle(path + 'ml_1m_test')
train_set = pd.read_pickle(path + 'ml_1m_train')

### Subset data

In [36]:
from Data_prep import leave_users_out
remaining, subset = leave_users_out(df, 40000)
print(len(subset))
df = subset
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes

351771


---
# Data Prep

## Dataset Params

In [8]:
val_perc = test_perc = 0.1
n_last_items_val = n_last_items_test = 1

total_items = len(df.item_id.unique())
total_users = len(df.user_id.unique())

## Create Split

In [9]:
from Data_prep import train_val_test_split
datasets = train_val_test_split(df, val_perc, test_perc, n_last_items_val, n_last_items_test)
train_set, val_set, test_set = datasets

---
# Neural Collaborative Filtering (NCF)
Using the NCF framework we build Generalized Matrix Factorisation (GMF), Multiplayer Perceptron Matrix Factorisation (MLP) and combine the two in Neural Matrix Factorisation (NeuMF)
- paper: http://papers.www2017.com.au.s3-website-ap-southeast-2.amazonaws.com/proceedings/p173.pdf
- blog: https://medium.com/@victorkohler/collaborative-filtering-using-deep-neural-networks-in-tensorflow-96e5d41a39a1
- code: https://github.com/Leavingseason/NeuralCF/blob/master

## Set Parameters

In [10]:
run_num = 'am_40k_GS_'
# run_num = 'ml_1m_nolf_8'

In [11]:
# GMF_params = {
#     'learning_rate': 0.001,
#     'batch_size': 256,
#     'nolf': 8,
#     'regs': [0.000001,0.000001], #ML: [0.000001,0.000001]
#     'epochs': 20,
#     'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
#     'num_neg': 4,
#     'ckpt_dir': f'../NeuMF_storage/GMF_ckpts_{run_num}/ckpts',
#     'optimizer':'Adam'
# }

In [12]:
# MLP_params = {
#     'learning_rate': 0.001,
#     'batch_size': 256,
#     'layers': [16,32,16,8],
#     'reg_layers': [0.0001,0,0,0],   #ML: [+-0.0005,0,0,0],
#     'epochs': 20,
#     'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
#     'num_neg': 4,
#     'ckpt_dir': f'../NeuMF_storage/MLP_ckpts_{run_num}/ckpts',
#     'optimizer':'Adam'
# }

In [15]:
NeuMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'layers': [16,32,16,8],
    'reg_layers': [0.0001,0,0,0],
    'reg_mf': [0.000001,0.000001],
    'nolf': 8,
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/NeuMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

## Init

In [21]:
from NCF import NCF
NCF = NCF(total_users, total_items, NeuMF_params=NeuMF_params)
NCF.build_NeuMF_model()

## Load samples

In [ ]:
# sample_path = '../NeuMF_storage/samples_ml_1m/'
# sample_name = 'ml_1m_sample_'

sample_path = '../NeuMF_storage/samples_am_40k/'
sample_name = 'am_40k_sample_'

In [ ]:
%%time
import csv
samples = []
pbar = progressbar.ProgressBar()
for sample_num in pbar(range(20)):
    with open(f'{sample_path}{sample_name}{sample_num}.csv', 'r') as read_obj:
        sample = [[], [], []]
        csv_reader = csv.reader(read_obj)
        sample_str = list(csv_reader)
        
        for user_input, item_input, label in zip(sample_str[0], sample_str[1], sample_str[2]):
            sample[0].append(int(user_input))
            sample[1].append(int(item_input))
            sample[2].append(int(label))
        
    samples.append(sample)

---
# Grid Search: NeuMF

## Parameter Space

In [27]:
learning_rates = [0.005, 0.001, 0.0001]
batch_sizes = [256, 512, 1024]
gmf_regs = [[0.000001, 0.000001], [0.00001, 0.00001], [0.0001, 0.0001]]
mlp_regs = [[0.000001, 0.000001, 0.000001, 0.000001], 
            [0.00001, 0.00001, 0.00001, 0.00001], 
            [0.0001, 0.0001, 0.0001, 0.0001],
            [0.001, 0.001, 0.001, 0.001],
            [0.000001, 0, 0, 0],
            [0.00001, 0, 0, 0],
            [0.0001, 0, 0, 0],
            [0, 0, 0, 0.0001],
            [0, 0, 0, 0.00001],
            [0, 0, 0, 0.000001]]
            
total_runs = len(learning_rates) * len(batch_sizes) * len(gmf_regs) * len(mlp_regs)
total_runs

270

### Init eval params

In [ ]:
from Evaluation import get_metrics
steps = 5
rank_at = 20
sample_len = 100

## Init store results

In [51]:
path = '../NeuMF_storage/GS/'
gs_result_name = 'GS_test'
storage_columns = ['tot_val_rec'] + ['val_metrics'] + list(NeuMF_params.keys())
gs_results = pd.DataFrame(columns=storage_columns)
gs_results.to_pickle(path + gs_result_name)

## Run Grid Search

In [ ]:
all_
for learning_rate in learning_rates:
    NCF.NeuMF_params['learning_rate'] = learning_rate
    
    for batch_size in batch_sizes:
        NCF.NeuMF_params['batch_size'] = batch_size
        
        for gmf_reg in gmf_regs:
            NCF.NeuMF_params['reg_mf'] = gmf_reg
            
            for mlp_reg in mlp_regs:
                NCF.NeuMF_params['reg_layers'] = mlp_reg
                
                
                

---
## NeuMF

### Full set scores

In [ ]:
%%time
NeuMF_ranked_df_full = NCF.get_predictions('NeuMF', test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(NeuMF_ranked_df_full, steps, rank_at, stats=False)

In [40]:
metrics_all_sample.to_pickle('../results/NCF/NCF_metrics_all_sample_am_40k')

### Sample scores

In [ ]:
%%time
NeuMF_ranked_df = NCF.sample_prediction('NeuMF', train_set, test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(NeuMF_ranked_df, steps, rank_at, stats=False)